In [1]:
import numpy as np
import skeliner as sk

# Automatic vs Manual Post-Processing

Recreate the automatic post-processing pipeline produced by `skeliner.skeletonize` step by step so you can tweak individual stages and understand what they do.


## Workflow
- Load a mesh segment and run the built-in automatic pipeline
- Disable post-processing and repeat each stage manually
- Compare the automatic and manual outputs to confirm they match


## Load the mesh segment
Provide the same segment identifier used in the automatic pipeline so both runs operate on identical geometry.


In [2]:
seg_id = 720575940545220339
MESH_PATH = f"../data/{seg_id}.obj"
mesh = sk.io.load_mesh(MESH_PATH)

## Automatic reference skeleton
We keep `postprocess=True` so that `skeliner` executes the entire pipeline for us. The result will be our ground truth for comparison.


In [3]:
skel_auto = sk.skeletonize(
    mesh,
    unit="nm",
    id=seg_id,
    verbose=True,
    postprocess=True,
)
print(f"Automatic skeleton nodes: {len(skel_auto.nodes)}")

[skeliner] starting skeletonisation (636,684 vertices, 1,274,227 faces)
 ↳  build surface graph                          … 6.50 s
 ↳  bin surface vertices by geodesic distance    … 6.36 s
 ↳  compute bin centroids and radii              … 13.51 s
 ↳  post-skeletonization soma detection          … 0.18 s
      └─ Found soma at [951560.0, 1086484.6, 34101.5]
      └─ (r = 10148.5, 5344.6, 2575.0)
 ↳  map mesh faces to skeleton edges             … 1.59 s
 ↳  merge redundant near-soma nodes              … 2.47 s
      └─ 129 nodes merged into soma
      └─ Moved soma to [951519.2, 1086418.2, 34093.4]
      └─ (r =  9589.7, 6004.5, 2608.4)
 ↳  bridge skeleton gaps                         … 0.46 s
 ↳  build global minimum-spanning tree           … 0.23 s
 ↳  prune tiny neurites                          … 2.24 s
      └─ Merged 8 peri-soma nodes into soma 
      └─ Moved soma to [951589.5, 1086350.8, 34190.9]
      └─ (r =  8795.8, 5831.4, 2727.5)
TOTAL (soma + core + post)                   

## Manual post-processing overview
We now disable the automatic stage and reproduce it ourselves:
1. Detect the soma region
2. Merge near-soma / fat nodes
3. Bridge disconnected components
4. Rebuild the global MST
5. Prune soma-adjacent neurites


In [4]:
skel = sk.skeletonize(
    mesh,
    unit="nm",
    id=seg_id,
    postprocess=False,
)
mesh_vertices = np.asarray(mesh.vertices, dtype=np.float64)
print(f"Manual pipeline starting nodes: {len(skel.nodes)}")


Manual pipeline starting nodes: 12168


### Stage 1 – Detect the soma
Identify the soma so that downstream steps know which nodes can be merged or pruned.


In [5]:
skel = skel.detect_soma(
    soma_radius_percentile_threshold=99.9,
    soma_radius_distance_factor=4.0,
    soma_min_nodes=3,
    mesh_vertices=mesh_vertices,
)


[skeliner.post]  post-skeletonization soma detection    … 0.20 s
      └─ Found soma at [951560.0, 1086484.6, 34101.5]
      └─ (r = 10148.5, 5344.6, 2575.0)


### Stage 2 – Merge near-soma nodes
Collapse nodes that lie inside or very close to the soma. This replicates the automatic fat-node cleanup.


In [6]:
skel = skel.merge_near_soma_nodes(
    mesh_vertices=mesh_vertices,
    inside_tol=0.0,
    near_factor=1.2,
    fat_factor=0.20,
    verbose=True,
)


[skeliner.post] merge redundant near-soma nodes         … 1.42 s
      └─ 129 nodes merged into soma
      └─ Moved soma to [951519.2, 1086418.2, 34093.4]
      └─ (r =  9589.7, 6004.5, 2608.4)


### Stage 3 – Bridge disconnected components
Fill small gaps between fragments before rebuilding the spanning tree. `bridge_gaps` operates in-place.


In [7]:
skel.bridge_gaps(
    bridge_max_factor=None,        # adaptive heuristic
    bridge_recalc_after=None,
    rebuild_mst=False,             # rebuild happens in the next stage
)


[skeliner.post] bridge skeleton gaps                    … 0.43 s
      └─ added 21 synthetic bridges


### Stage 4 – Rebuild the MST
Recompute the minimum spanning tree now that gaps are bridged.


In [8]:
skel = skel.rebuild_mst(verbose=True)


[skeliner.post] build global minimum-spanning tree      … 0.33 s
      └─ edges contracted 12039 → 12038


### Stage 5 – Prune soma-adjacent neurites
Drop tiny neurites that hug the soma to match the automatic pruning behavior.


In [9]:
skel = skel.prune_neurites(
    mesh_vertices=mesh_vertices,
    tip_extent_factor=1.2,
    stem_extent_factor=3.0,
    drop_single_node_branches=True,
)

[skeliner.post] prune tiny neurites                     … 2.31 s
      └─ Merged 8 peri-soma nodes into soma 
      └─ Moved soma to [951589.5, 1086350.8, 34190.9]
      └─ (r =  8795.8, 5831.4, 2727.5)


## Validate manual vs automatic results
The manual pipeline should be identical to the automatic one (up to floating point noise). We compare node counts and coordinates.


In [10]:
manual_nodes = len(skel.nodes)
auto_nodes = len(skel_auto.nodes)
print(f"Manual nodes: {manual_nodes}\nAuto nodes:   {auto_nodes}")

nodes_match = np.allclose(skel.nodes, skel_auto.nodes)
print(f"Node coordinates identical: {nodes_match}")
assert nodes_match, "Manual pipeline diverged from automatic" 


Manual nodes: 11558
Auto nodes:   11558
Node coordinates identical: True


In [11]:
%load_ext watermark
%watermark --time --date --timezone --updated --python --iversions --watermark

Last updated: 2025-11-13 15:55:15CET

Python implementation: CPython
Python version       : 3.13.9
IPython version      : 9.7.0

numpy   : 2.3.4
skeliner: 0.2.3

Watermark: 2.5.0

